### please work with a copy of this (or revert before git commit) to avoid git commit conflicts!
the analysis requires a few parameters:
- `path`: path to a data file output from the power sweep acquisition flowgraph
- `holdoff`: "transient window" duration to ignore at the start of each frequency step

In [1]:
%pylab inline

# permits 'import read' from the parent directory
import sys
if '..' not in sys.path:
    sys.path.insert(1,'..')
import read
import os
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


### Average power data
The function `read.swept_average` reads data in various format versions output by GNURadio acquisition dating back to early April 2020.

Given the path to a `.dat` file, it reads
1. the corresponding raw binary data, and
2. additional metadata about the acquisition from the corresponding `.yaml` file.

In [49]:
raw = np.fromfile(r'../data/dkuester cold noise 2020-05-28_13h14m48s.swept_power.dat', dtype='float32')
raw[2::1500]

array([6.1100000e+08, 2.0370240e-07, 1.9614643e-07, 2.0023582e-07,
       2.1655353e-07, 2.1672099e-07, 2.0765303e-07, 2.0379002e-07,
       1.9723062e-07, 2.0032633e-07, 2.0602367e-07, 2.5954367e-07,
       2.5793483e-07, 2.6321544e-07, 2.7840443e-07, 2.3358561e-07,
       2.2094716e-07, 2.4879935e-07, 3.5153215e-07, 2.1648964e-07,
       2.2391750e-07, 2.3800804e-07, 2.2446059e-07, 2.3343209e-07,
       2.2611314e-07, 2.2116970e-07, 2.4305268e-07, 2.2844142e-07,
       2.6069685e-07, 2.1997749e-07], dtype=float32)

In [40]:
spectrum, metadata = read.swept_average(r'../data/dkuester cold noise 2020-05-28_13h14m48s.swept_power.dat')

# its convenient to make the metadata information into variables here
globals().update(metadata)

# summarize the metadata
for k, v in metadata.items():
    if not isinstance(k, float) and k != 'peak_sample':
        print(f'{k}: {v}')

0.25 0.0005 500
aperture_time: 0.0005
dwell_time: 0.25
gain: 15.0
sample_rate: 4000000.0
start_timestamp: 1590693293.1665
center_frequencies: [611000000.0, 701500000.0, 709000000.0, 734000000.0, 739000000.0, 751000000.0, 782000000.0, 821300000.0, 842500000.0, 866300000.0, 887500000.0, 2412000000.0, 2437000000.0, 2462000000.0, 2695000000.0, 5170000000.0, 5190000000.0, 5210000000.0, 5230000000.0, 5240000000.0, 5775000000.0, 5795000000.0]


The output data in the `spectrum` variable is in the form of a pandas `DataFrame` object. It packages together the raw power levels in each aperture with information on the time and frequency axes, as well as the sweep number in which it was acquired. Jupyter notebooks display a formatted summary in cells that end with a `DataFrame` object.

In [41]:
spectrum

Aperture power sample                                         0    \
Sweep Time                                Frequency                 
0     2020-05-28 13:14:08-06:00           611.0      1.924221e-07   
      1969-12-31 17:00:00.000000199-07:00 0.0        1.985666e-07   
      1969-12-31 17:00:00.000000193-07:00 0.0        2.090586e-07   
      1969-12-31 17:00:00.000000200-07:00 0.0        1.995243e-07   
      1969-12-31 17:00:00.000000214-07:00 0.0        2.095670e-07   
...                                                           ...   
3     1969-12-31 17:00:00.000000215-07:00 0.0        2.201287e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.235840e-07   
      1969-12-31 17:00:00.000000251-07:00 0.0        2.374181e-07   
      1969-12-31 17:00:00.000000233-07:00 0.0        2.210473e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.231219e-07   

Aperture power sample                                         1    \
Sweep Time                                Frequency                 
0     2020-05-28 13:14:08-06:00           611.0      1.987959e-07   
      1969-12-31 17:00:00.000000199-07:00 0.0        2.033834e-07   
      1969-12-31 17:00:00.000000193-07:00 0.0        2.047118e-07   
      1969-12-31 17:00:00.000000200-07:00 0.0        2.061798e-07   
      1969-12-31 17:00:00.000000214-07:00 0.0        2.124070e-07   
...                                                           ...   
3     1969-12-31 17:00:00.000000215-07:00 0.0        2.242016e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.134672e-07   
      1969-12-31 17:00:00.000000251-07:00 0.0        2.341506e-07   
      1969-12-31 17:00:00.000000233-07:00 0.0        2.354151e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.602130e-07   

Aperture power sample                                         2    \
Sweep Time                                Frequency                 
0     2020-05-28 13:14:08-06:00           611.0      2.151672e-07   
      1969-12-31 17:00:00.000000199-07:00 0.0        1.984256e-07   
      1969-12-31 17:00:00.000000193-07:00 0.0        2.084875e-07   
      1969-12-31 17:00:00.000000200-07:00 0.0        2.099680e-07   
      1969-12-31 17:00:00.000000214-07:00 0.0        2.121581e-07   
...                                                           ...   
3     1969-12-31 17:00:00.000000215-07:00 0.0        2.401122e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.337594e-07   
      1969-12-31 17:00:00.000000251-07:00 0.0        2.411773e-07   
      1969-12-31 17:00:00.000000233-07:00 0.0        2.301517e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.327364e-07   

Aperture power sample                                         3    \
Sweep Time                                Frequency                 
0     2020-05-28 13:14:08-06:00           611.0      1.989873e-07   
      1969-12-31 17:00:00.000000199-07:00 0.0        2.048999e-07   
      1969-12-31 17:00:00.000000193-07:00 0.0        1.987665e-07   
      1969-12-31 17:00:00.000000200-07:00 0.0        2.184851e-07   
      1969-12-31 17:00:00.000000214-07:00 0.0        2.041775e-07   
...                                                           ...   
3     1969-12-31 17:00:00.000000215-07:00 0.0        2.495015e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.219228e-07   
      1969-12-31 17:00:00.000000251-07:00 0.0        2.447461e-07   
      1969-12-31 17:00:00.000000233-07:00 0.0        2.388229e-07   
      1969-12-31 17:00:00.000000238-07:00 0.0        2.167003e-07   

Aperture power sample                                         4    \
Sweep Time                                Frequency                 
0     2020-05-28 13:14:08-06:00           611.0      2.031474e-07   
      1969-12-31 17:00:00.000000199-07:00 0.0        2.009972e-07   
      1969-12-31 17:00:00.000000193-07:00 0.0        2.144696e-07   
      1969-12-31 17:00:00.000000200-07:00 0.0        1.992823e-

You can also get and use the aperture power and time and frequency index data as `numpy` arrays:

In [42]:
times = spectrum.index.get_level_values('Time').values.astype('str') # or: .values.astype('float32') for UNIX time (seconds since ~1970)
frequency = spectrum.index.get_level_values('Frequency').values
sweep = spectrum.index.get_level_values('Sweep').values
power = spectrum.values

print('frequency: ', frequency)
print('power: ', power)
print('times: ', times)

frequency:  [611.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.]
power:  [[1.9242215e-07 1.9879593e-07 2.1516716e-07 ... 2.0872289e-07
  2.1035481e-07 2.0702819e-07]
 [1.9856660e-07 2.0338335e-07 1.9842561e-07 ... 1.5906932e+09
  7.0150003e+08 3.3731617e-06]
 [2.0905863e-07 2.0471181e-07 2.0848755e-07 ... 2.0191742e-07
  1.9815148e-07 2.0250209e-07]
 ...
 [2.3741809e-07 2.3415058e-07 2.4117730e-07 ... 2.4228811e-07
  2.1511647e-07 2.3926862e-07]
 [2.2104732e-07 2.3541511e-07 2.3015167e-07 ... 2.3178056e-07
  2.3754546e-07 2.3378028e-07]
 [2.2312190e-07 2.6021297e-07 2.3273635e-07 ... 2.4121732e-07
  2.4

### Sort the data
For many tasks it's useful to index by Frequency. This is easy to do with `sort_index`:

In [ ]:
spectrum = spectrum.sort_index(level=['Frequency', 'Time'])
spectrum.head(10)

### Indexing the python data
There are 3 index axes:
1. The sweep number
2. The start time of the dwell window
3. The RF center frequency in this dwell window.

To access subsets of the data within `spectrum`, we index on the value of this axis. For example, to slice only the data 2695 MHz:

In [ ]:
spectrum.loc[pd.IndexSlice[:,:,2695.],:].head()

you can also specify ranges along each axis similarly

In [ ]:
spectrum.loc[pd.IndexSlice[:,'2020-05-20 16:18:00':'2020-05-20 16:19:00',2695.],:].head()

In [ ]:
spectrum.loc[pd.IndexSlice[2:4,:,2695.],:].head()